# Imports

In [ ]:
!pip install -q -U google-generativeai 
!pip install --upgrade transformers datasets[audio] accelerate 

In [ ]:
import os
import time
import numpy as np
import pandas as pd
from PIL import Image
import soundfile as sf
from json import loads
import seaborn as sns 
import matplotlib.pyplot as plt
import typing_extensions as typing
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# Data Loading

**Defining Model** 

In [ ]:
import google.generativeai as genai 
os.environ["API_KEY"] = ""
genai.configure(api_key=os.environ["API_KEY"]) 

In [ ]:
model = genai.GenerativeModel("gemini-2.5-flash", system_instruction="You are an expert content moderator.") 

In [ ]:
class LLM_Output(typing.TypedDict):
    label: str
    language: list[str]
    response: str

**Prompt** 

In [ ]:
prompt =  """"A video can be considered inappropriate for children if it contains physical violence (cartoonish, realistic, or gory), interpersonal violence (bullying, pranks, meanness, belittling, controlling behavior, talking down to others, or manipulation), self-harm or suicide (depictions of harm inflicted on oneself or suicidal thoughts/tendencies), extreme stunts (life-endangering, high-risk activities/challenges that require adult supervision), dangerous products or services (like paintball, airsoft, fireworks, weapons, hunting equipment, graffiti products), scary content (horror, suspense, loud violence, zombies, skeletons, masks, scary clowns, blood, dangerous fire, car crashes, medical procedures or other scary visuals), sexual content (innuendos, sexual behavior, nudity, suggestiveness), intimacy (romantic conduct, couples expressing love, sensual contact, dating, relationships or any expression of love or loss for a significant other,  lyrics about or mentions of love or romance, heartbreak), inappropriate clothing (significant skin exposure or sheer clothing outside of appropriate contexts such as sunbathing, swinmwear,etc), substance use (mention, depictions, or use of alcohol, drugs, tobacco), gambling (betting, gambling, casinos, online or real world casino games, online trading), offensive language (from mild to severe with words such as “stupid” or “idiot”), stereotypes (racial, ethnic, gender, body shaming), crude humor (inappropriate humor, innuendos), weapons (guns, swords, knives, etc but toy guns such as nerf guns are fine), unsuitable media (ads for violent or adult-themed video games, casino games, games with romance or battle or first person-shooter games), fight sports (boxing, wrestling, martial arts), financial content (difference, rolling spot forex, financial spread betting and synonymous products but banking ads are fine), health and beauty services (cosmetic procedures, weight loss, tanning, piercings, tattoos, cosmetics and makeup (no matter their essential benefits) and other products related to external personal care focused on body image but ads on fashion products and essential hygiene products like sunscreen, shampoo, toothpaste, etc. are fine, however, products like sanitary care and hair removal is inappropriate), ads or sponsored content for consumable food or drink products (regardless of nutritional content), where the product is consumed by a person or shown with branding, packaging, or promotional intent are inappropriate; general content like cooking shows, recipes, or scenes where food is simply present or being eaten without promotion are acceptable, pet/animal food products and healthcare, healthcare or medical topics (reproductive health, mental health, substance abuse, eating disorders, pharmaceuticals, health insurance, any nutritional or dietary supplements), sensitive topics (politics, religion, controversial social issues, astrology, occult, paranormal, religious sermons), privacy or safety risks (services or competitions requiring personal information, social networks for strangers, spy services, or video game skin trading), or any explicit depictions or discussions of death or dying. Consider that you are an expert content moderator. Based on the provided title, description, channel title, tags, thumbnail and audio, is the video advertisement using this auido inappropriate for children on this definition? If it is, label it as “inappropriate.” If it is not, label it as “appropriate.” Justify your conclusion. Also, mention what language is being spoken if there is any. 
Make the outputs in JSON format: {“label”: “appropriate” or “inappropriate”, “justification”: “brief explanation of the classification”, “language”: “language detected in the video”}."""
prompt 

**Running Model on Dataset** 

In [ ]:
import pandas as pd
import requests
import json
import time
import os

# === Parameters ===
start_idx = 0
end_idx = 1153
TEMP_THUMB_PATH = "/kaggle/working/temp_thumb.jpg"
AUDIO_BASE_PATH = "/kaggle/input/youtube-data/Ads/Ads"  # Adjust path if needed

# === Load metadata CSV (includes Video Id, title, tags, labels, etc.) ===
metadata_df = pd.read_csv('/kaggle/input/metadata/ad_metadata_filtered.csv')
metadata_df["Video Id"] = metadata_df["id"].astype(str)

# === Slice the range ===
sliced_df = metadata_df.iloc[start_idx:end_idx]
print(f"Processing rows: {len(sliced_df)}")

# === Output containers ===
ids = []
predicted_labels = []
languages = []
responses = []
ground_truths_ = []
remaining = []

# === Process each ad ===
for i, row in sliced_df.iterrows():
    ad_id = row['Video Id']
    title = row.get('title', '')
    description = row.get('description', '')
    channel_title = row.get('channelTitle', '')
    tags = row.get('tags', '')
    label = row['Primary Label']

    # === Upload audio file ===
    audio_path = os.path.join(AUDIO_BASE_PATH, ad_id, "audio.mp3")
    try:
        audio_file = genai.upload_file(path=audio_path, resumable=False)
    except Exception as e:
        print(f"Error uploading audio for {ad_id}: {e}")
        remaining.append(ad_id)
        continue

    # === Attempt to download and upload thumbnail ===
    thumbnail_url = f"https://i.ytimg.com/vi/{ad_id}/default.jpg"
    thumbnail_file = None
    try:
        img_response = requests.get(thumbnail_url)
        if img_response.status_code == 200:
            with open(TEMP_THUMB_PATH, "wb") as f:
                f.write(img_response.content)
            thumbnail_file = genai.upload_file(path=TEMP_THUMB_PATH)
        else:
            print(f"Thumbnail not found for {ad_id}, continuing without thumbnail.")
    except Exception as e:
        print(f"Error fetching thumbnail for {ad_id}: {e}, continuing without thumbnail.")

    # === Prepare metadata JSON ===
    json_input = {
        "title": title,
        "description": description,
        "channel_title": channel_title,
        "tags": tags
    }

    # === Build Gemini input ===
    inputs = [audio_file]
    if thumbnail_file:
        inputs.append(thumbnail_file)
    inputs.append({"text": json.dumps(json_input, indent=2)})
    inputs.append({"text": prompt})

    # === Run inference ===
    try:
        response = model.generate_content(
            inputs,
            generation_config=genai.GenerationConfig(
                response_mime_type="application/json",
                response_schema=LLM_Output
            ),
            safety_settings={
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
            }
        )
    except Exception as e:
        print(f"Error during inference for {ad_id}: {e}")
        remaining.append(ad_id)
        continue

    # === Parse response ===
    try:
        dictionary = json.loads(response.text)
        print(f"Completed: {i} {ad_id} | Label: {dictionary['label']}  | Primary Label: {label}")

        ids.append(ad_id)
        predicted_labels.append(dictionary['label'])
        languages.append(dictionary['language'])
        responses.append(dictionary['response'])
        ground_truths_.append(label)
        time.sleep(20)

    except Exception as e:
        print(f"Error parsing response for {ad_id}: {e}")
        remaining.append(ad_id)
        time.sleep(20)
        continue

# === Summary ===
print("\nRemaining (errors):", remaining)

In [ ]:
# At the end, print remaining videos 
remaining_df = pd.DataFrame({'Video Id': remaining})

# Save to CSV
remaining_df.to_csv(f'remaining-{start_idx}-{end_idx}.csv', index=False)

print("Remaining videos saved to remaining.csv")
print("Remaining videos with errors:", remaining) 

In [ ]:
for i in range(len(responses)): 
    print('True Label: ', ground_truths_[i], '\tPrediction: ', predicted_labels[i]) 

In [ ]:
new_df = pd.DataFrame({
    'Video Id': ids,
    'Primary Label': ground_truths_,
    'Predicted Label': predicted_labels,
    'Response': responses, 
    'Languages': languages 
})

new_df.head() 

In [ ]:
new_df.to_csv(f'/kaggle/working/gemini-audio-metadata-{start_idx}-{end_idx}.csv', index=False)

In [ ]:
# Changing to binary lists 

predictions = [1 if pred == 'inappropriate' else 0 for pred in predicted_labels] 
ground_truths = [1 if label == 'inappropriate' else 0 for label in ground_truths_] 

In [ ]:
# Obtaining classification report 
from sklearn.metrics import classification_report 

report = classification_report(ground_truths, predictions) 
print(report) 

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ground_truths, predictions)

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Appropriate', 'Inapproriate'], yticklabels=['Appropriate', 'Inapproriate'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()